============================
## MTA Turnstile Data
 <i> Cleaning & Structuring</i> 
 
============================

by Sarah Weinflash


I downloaded the data from the following [source](https://data.ny.gov/Transportation/Turnstile-Usage-Data-2015/ug6q-shqc) as a CSV file. It includes the following variables:
- <b>C/A, Unit, and SCP:</b> unique identifiers for the turnstiles
- <b>Station:</b> subway station at which the turnstile is located
- <b>Line Name:</b> subway lines available at that station
- <b>Date and Time:</b> date and time that the data was pulled
- <b>Entries and Exits:</b> cumulative number of entries and exists respectively, collected by the turnstiles

Below, you can see my code for importing the necessary libraries as well as the data. 

In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import matplotlib.pyplot as plt

In [4]:
data = pd.read_csv('/Users/sarahweinflash/Desktop/Education/CIS 9655 - Data Visualization/Presentation 3/Original_Data.csv', index_col=None)
data = data[['C/A','Unit','SCP','Station','Line Name','Date','Time','Entries','Exits                                                     ']]
data.head()

/var/folders/m_/bn7_5xgd5k78dxpv8nsvb1r00000gn/T/ipykernel_26853/3064591384.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/Users/sarahweinflash/Desktop/Education/CIS 9655 - Data Visualization/Presentation 3/Original_Data.csv', index_col=None)


,C/A,Unit,SCP,Station,Line Name,Date,Time,Entries,Exits
0,A002,R051,02-00-00,59 ST,NQR456,01/01/2016,23:00:00,5478127,1849126
1,A002,R051,02-00-00,59 ST,NQR456,01/01/2016,19:00:00,5477962,1849103
2,A002,R051,02-00-00,59 ST,NQR456,01/01/2016,15:00:00,5477676,1849053
3,A002,R051,02-00-00,59 ST,NQR456,01/01/2016,11:00:00,5477548,1848989
4,A002,R051,02-00-00,59 ST,NQR456,01/01/2016,07:00:00,5477508,1848946


<b> Data Types </b>

- I renamed the 'Entries' and 'Exits' columns to indicate that they are reporting <i>cumulative</i> data.
- Some of the line names are entirely numerical, so I ensured that Python was reading them as strings.
- Date and Time were strings; I reclassified them as Dates and Times respectively, and then combined them into a single DateTime variable.

In [5]:
data = data.rename(columns = {'Exits                                                     ':'Cum_Exits', 'Entries':'Cum_Entries'})
data['Line Name'] = data['Line Name'].apply(lambda x: str(x))
data['Date'] = data['Date'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y').date())
data['Time'] = data['Time'].apply(lambda x: datetime.strptime(x, '%H:%M:%S').time())
data['DateTime'] = [datetime.combine(data['Date'][i], data['Time'][i]) for i in range(0,len(data['Time']))]
data.head()

,C/A,Unit,SCP,Station,Line Name,Date,Time,Cum_Entries,Cum_Exits,DateTime
0,A002,R051,02-00-00,59 ST,NQR456,2016-01-01,23:00:00,5478127,1849126,2016-01-01 23:00:00
1,A002,R051,02-00-00,59 ST,NQR456,2016-01-01,19:00:00,5477962,1849103,2016-01-01 19:00:00
2,A002,R051,02-00-00,59 ST,NQR456,2016-01-01,15:00:00,5477676,1849053,2016-01-01 15:00:00
3,A002,R051,02-00-00,59 ST,NQR456,2016-01-01,11:00:00,5477548,1848989,2016-01-01 11:00:00
4,A002,R051,02-00-00,59 ST,NQR456,2016-01-01,07:00:00,5477508,1848946,2016-01-01 07:00:00


<b> Data Aggregation & Structuring </b>

- I will be attempting to calculate the number of entries and exits in a given time period (rather than cumulative)
- I set the index as the unique identifiers for the turnstiles, and then dropped any duplicate values
- I dropped the index in order to group by the unique turnstiles, aggregating by sum
- To ensure that the cumulative values are in the order in which they were reported, I sorted the values by the DateTime variable. Later, this will ensure I am taking the difference of the timestamp immediately before
- I reset the index as the unique identifiers for the turnstiles, which will ensure that I am finding the difference in cumulative entries/exits of the same turnstile

In [6]:
data = data.reset_index(drop=True)
data2 = data.set_index(['C/A', 'Unit', 'SCP'])
data2 = data2.drop_duplicates()
data2 = data2.reset_index(drop=False)
data2 = data2.groupby(['C/A', 'Unit', 'SCP','DateTime', 'Line Name', 'Station']).agg('sum')
data2 = data2.reset_index(drop=False)
data2 = data2.sort_values(by=['DateTime'])
data2 = data2.set_index(['C/A','Unit','SCP'])
data2.head()

/var/folders/m_/bn7_5xgd5k78dxpv8nsvb1r00000gn/T/ipykernel_26853/1350613069.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data2 = data2.groupby(['C/A', 'Unit', 'SCP','DateTime', 'Line Name', 'Station']).agg('sum')


DateTime Line Name        Station  Cum_Entries  \
C/A   Unit SCP                                                         
TRAM1 R468 00-05-00 2015-01-01         R  RIT-MANHATTAN          811   
N531  R129 01-00-00 2015-01-01        FG      BERGEN ST      1159048   
N530  R301 00-00-02 2015-01-01         F        YORK ST      7787316   
           00-00-01 2015-01-01         F        YORK ST     10336286   
           00-00-00 2015-01-01         F        YORK ST     10215474   

                     Cum_Exits  
C/A   Unit SCP                  
TRAM1 R468 00-05-00          0  
N531  R129 01-00-00    2030935  
N530  R301 00-00-02    1257701  
           00-00-01    5634922  
           00-00-00   19570448

<b> New Variables </b>
- I found the lapsed time between data collection by ensuring that the two entries had the same index (and thus, represented the same turnstile); then, I took the difference between the two values and named the variable "Lapsed Time". All first values in the dataset were labeled as null. 
- Using the same technqiue( I found the difference in cumulative entries and exits, thus generating the total number of entries/exits between data collection times.
- I removed any null values of Lapsed Time in order to proceed with the rate calculations.
- Because the lapsed time was not consistent, I calculated the entry and exit rate by dividing the number of entries and exits by the lapsed time.

In [7]:
Lapsed_Time = [None]

for i in range(1,len(data2)):
    if data2.index[i] == data2.index[i-1]:
        Lapsed_Time.append(pd.to_timedelta(data2['DateTime'][i] - data2['DateTime'][i-1]))
    else:
        Lapsed_Time.append(None)
data2['Lapsed_Time'] = Lapsed_Time
data2.head()

DateTime Line Name        Station  Cum_Entries  \
C/A   Unit SCP                                                         
TRAM1 R468 00-05-00 2015-01-01         R  RIT-MANHATTAN          811   
N531  R129 01-00-00 2015-01-01        FG      BERGEN ST      1159048   
N530  R301 00-00-02 2015-01-01         F        YORK ST      7787316   
           00-00-01 2015-01-01         F        YORK ST     10336286   
           00-00-00 2015-01-01         F        YORK ST     10215474   

                     Cum_Exits Lapsed_Time  
C/A   Unit SCP                              
TRAM1 R468 00-05-00          0         NaT  
N531  R129 01-00-00    2030935         NaT  
N530  R301 00-00-02    1257701         NaT  
           00-00-01    5634922         NaT  
           00-00-00   19570448         NaT

In [ ]:
Entry = [None]
Exit = [None]

for i in range(1, len(data2)):
    if (data2.index[i] == data2.index[i-1]):
        Entry.append(abs(data2['Cum_Entries'][i] - data2['Cum_Entries'][i-1]))
        Exit.append(abs(data2['Cum_Exits'][i] - data2['Cum_Exits'][i-1]))
    else:
        Entry.append(data2['Cum_Entries'][i])
        Exit.append(data2['Cum_Exits'][i])
data2['Entries'] = Entry
data2['Exits'] = Exit
data2.head()

In [ ]:
data2 = data2[data2['Lapsed_Time'].notnull()]

In [ ]:
data2[data2['Lapsed_Time'] == data2['Lapsed_Time'].max()]
data2['Lapsed_Time'].mean()

In [ ]:
Entry_Rate = [None]
Exit_Rate = [None]

for i in range(1,len(data2)):

    if (data2.index[i] == data2.index[i-1]):
        a = data2['Lapsed_Time'][i].total_seconds()
        b = data2['Entries'][i]/a
        
        c = data2['Lapsed_Time'][i].total_seconds()
        d = data2['Exits'][i]/c
        
    else:
        b = None
        d = None
        
    Entry_Rate.append(b)
    Exit_Rate.append(d)
    
data2['Entry_Rate'] = Entry_Rate
data2['Exit_Rate'] = Exit_Rate
data2.head()

<b> Cleaning </b>
- I reset the index so that I could drop the index variables, which will not be useful for future analysis, as they provide no information beyond unique coding. 
- There were a few outliers in which the number of entries were abnormally large; I removed these rows. 

In [ ]:
data2 = data2.reset_index(drop = False)
data2 = data2[['Station', 'Line Name', 'DateTime','Lapsed_Time','Entries','Exits','Entry_Rate','Exit_Rate']]
data2 = data2[(data2['Entries']<100000)&(data2['Entry_Rate']<50)]
data2.head()

<b> Subway Lines </b>
- Because the subway lines were listed as strings, they provided very little information; '123' was different from '123NQR' was different from '23'. 
- I generated a list called "Lines" of numbers (as strings) 1 thru 7, and all letters A thru Z. 
- I turned this list into a DataFrame, with the list as the columns.
- Every unique iteration of 'Line Name' was added as rows of the DataFrame. The dataframe was populated by True/False based on whether the Line is present in the Line Name. 
- There are a handful of letters of the alphabet which are not Subway lines (H, X, K, etc.). I removed those columns from the DataFrame.
- Each train is represented in the 'Line Name' column as a single character; I used this fact to create a new column which calculated the number of trains in each station, by counting the length of the string in each row of 'Line Name'.
- I found analyzing the individual trains to be overwhelming; to give future analysts options in their exploration, I also grouped the trains by their colors, using a similar but shorter process as I used to generate the Lines originally. 
- Note that in this dataset, there is only N, Q, R, 4, 5, and 6 trains
- I merged the Lines DataFrame with the original dataset.

In [ ]:
Lines = [*range(1,8)]
Lines = list(map(str, Lines))
Lines.insert(0, 'Line Name')

for i in range(ord('a'), ord('z')+1):
    Lines.append(chr(i).upper())
Lines = pd.DataFrame(columns = Lines)

for i in data2['Line Name'].unique():
    Lines_List = []
    for j in Lines.columns:
        if str(j) in str(i):
            Lines_List.append(True)
        else:
            Lines_List.append(False)
    Lines_List[0] = i
    Lines.loc[len(Lines)] = Lines_List

for i in Lines.columns:
    if any(Lines[i]):
        continue
    else:
        Lines = Lines.drop(i, axis=1)
Lines

In [ ]:
Num_Trains = []
for i in data2['Line Name']:
    Num_Trains.append(len(i))
data2['Trains'] = Num_Trains
data2.head()

In [ ]:
blue = []
orange = []
brown = []
yellow = []
red = []
green = []
other = []

for i in Lines.index:
   
    if (Lines['A'][i] == True) | (Lines['E'][i] == True) | (Lines['C'][i] == True):
        blue.append(True)
    else:
        blue.append(False)

    if (Lines['B'][i] == True) | (Lines['D'][i] == True) | (Lines['F'][i] == True) | (Lines['M'][i] == True):
        orange.append(True)
    else:
        orange.append(False)

    if (Lines['J'][i] == True) | (Lines['Z'][i] == True):
        brown.append(True)
    else:
        brown.append(False)

    if (Lines['N'][i] == True) | (Lines['Q'][i] == True) | (Lines['R'][i] == True):
        yellow.append(True)
    else:
        yellow.append(False)

    if (Lines['1'][i] == True) | (Lines['2'][i] == True) | (Lines['3'][i] == True):
        red.append(True)
    else:
        red.append(False)
        
    if (Lines['4'][i] == True) | (Lines['5'][i] == True) | (Lines['6'][i] == True):
        green.append(True)
    else:
        green.append(False)     

    if (Lines['S'][i] == True) | (Lines['G'][i] == True) | (Lines['L'][i] == True) | (Lines['7'][i] == True):
        other.append(True)
    else:
        other.append(False)

Lines['ACE'] = blue
Lines['BDFM'] = orange
Lines['JZ'] = brown
Lines['NQR'] = yellow
Lines['123'] = red
Lines['456'] = green
Lines['Other'] = other
Lines

In [30]:
data3 = data2.merge(Lines, on='Line Name')
data3.head()

,Station,Line Name,DateTime,Lapsed_Time,Entries,Exits,Entry_Rate,Exit_Rate,Trains,4,5,6,N,Q,R,NQR,456
0,LEXINGTON AVE,NQR456,2015-07-08 04:00:00,0 days 04:00:00,0.0,0.0,0.000000,0.000000,6,True,True,True,True,True,True,True,True
1,LEXINGTON AVE,NQR456,2015-07-08 08:00:00,0 days 04:00:00,45.0,73.0,0.003125,0.005069,6,True,True,True,True,True,True,True,True
2,LEXINGTON AVE,NQR456,2015-07-08 12:00:00,0 days 04:00:00,183.0,284.0,0.012708,0.019722,6,True,True,True,True,True,True,True,True
3,LEXINGTON AVE,NQR456,2015-07-08 16:00:00,0 days 04:00:00,301.0,69.0,0.020903,0.004792,6,True,True,True,True,True,True,True,True
4,LEXINGTON AVE,NQR456,2015-07-08 20:00:00,0 days 04:00:00,901.0,92.0,0.062569,0.006389,6,True,True,True,True,True,True,True,True


<b> Final Steps </b>
- The dataset was extremely large; I created a subset of the dataset with 10,000 rows randomly selected. It is still a very large dataset and useful for analysis, but easier to process than the full dataset.
- I exported both the full and subsetted dataset to CSVs. 

In [31]:
subset_data = data3.sample(n=10000)
data3.to_csv('Cleaned_Dataset.csv')
subset_data.to_csv('subset_data.csv')